First example 

In [ ]:
import torch
import torch.nn as nn
from torchvision.models import resnet50


class SimpleDETR(nn.Module):
  def __init__(self, num_classes, hidden_dim, num_heads,
               num_enc_layers, num_dec_layers):
    
    super(SimpleDETR,self).__init__()
    self.num_classes = num_classes
    self.hidden_dim = hidden_dim
    self.num_heads = num_heads
    self.num_enc_layers = num_enc_layers
    self.num_dec_layers = num_dec_layers 

    self.backbone = nn.Sequential(
        *list(resnet50(pretrained=True).children())[:-2])

    self.conv = nn.Conv2d(2048, hidden_dim, 1)

    self.transformer = nn.Transformer(hidden_dim, num_heads,
                                      num_enc_layers, num_dec_layers)
    
    self.to_classes = nn.Linear(hidden_dim, num_classes+1)
    self.to_bbox = nn.Linear(hidden_dim, 4)

    self.object_query = nn.Parameter(torch.rand(100, hidden_dim))

    self.row_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
    self.col_embed = nn.Parameter(torch.rand(50, hidden_dim // 2))
    self.states = dict({'conv_features':None,'H':None,'W':None,
                        'pos_enc':None,'object_query':self.object_query,
                        'pred_classes':None,'pred_bboxes':None})
    
  def forward(self, X):
    X = self.backbone(X)
    h = self.conv(X)
    self.conv_features = h.data

    self.states['conv_features'] = h.data

    H, W = h.shape[-2:]
    self.states['H']=H
    self.states['W']=W
    

    pos_enc = torch.cat([
                         self.col_embed[:W].unsqueeze(0).repeat(H,1,1),
                         self.row_embed[:H].unsqueeze(1).repeat(1,W,1)
                         ],
                    dim=-1).flatten(0,1).unsqueeze(1)

    self.states['pos_enc'] = pos_enc.data
    
    h = self.transformer(pos_enc + h.flatten(2).permute(2,0,1),
                         self.object_query.unsqueeze(1))
        
    class_pred = self.to_classes(h)
    bbox_pred = self.to_bbox(h).sigmoid()

    self.states['pred_classes']=class_pred.detach().data
    self.states['pred_bbox']=bbox_pred.detach().data

    return class_pred, bbox_pred

In [ ]:
detr = SimpleDETR(1,256,8,6,6)
X = torch.rand(1,3,100,100)
cls, box = detr(X)
box.size()

#### Import Modules

In [ ]:
import torch
from torchvision import transforms as T

import requests # for loading images from web
from PIL import Image, ImageDraw # for viewing images and drawing bounding boxes

from IPython.display import display, Markdown, Latex
from termcolor import colored
import matplotlib.pyplot as plt

## Load the model from PyTorch


Here we use torchhub to load the pretrained model for object detection from the Facebook Research github repository. 

In [ ]:
model = torch.hub.load('facebookresearch/detr','detr_resnet50',pretrained=True)

if torch.cuda.is_available():
  torch.device('cuda')
  model = model.cuda()

## Download the Images

In [ ]:
# url="https://cdn-s-www.vosgesmatin.fr/images/34BE08B0-9DF7-4753-B2F2-797BBB22DAB3/NW_raw/la-vaisselle-est-presente-en-nombre-sur-ces-etageres-photo-vm-david-henry-1625678804.jpg'
# url = 'https://www.gannett-cdn.com/-mm-/bfb8a52de1198e08f1242088b975b76a6c440df0/c=0-244-4873-2992/local/-/media/Phoenix/Phoenix/2014/09/16/1410906318000-MOD-Pizza-Interior.jpg' 
url='https://www.francetvinfo.fr/pictures/sKpQSVq-5vvH_nG1MokT9VWpRqY/0x0:4608x2592/944x531/filters:format(webp)/2020/06/04/phpWlyXR2.jpg'

In [ ]:
im = Image.open(requests.get(url, stream=True).raw)
fig = plt.figure(figsize=(15,10))
plt.imshow(im)
plt.show()

### ImageTransforms



As a preprocessing step, we resize our image, convert it to a tensor, and then normalize the pixel values below:

In [ ]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
img_tensor = transform(im)

### Object Classes


We manually define the names of each COCO class, as well as the colors that we will use for each bounding box.

In [ ]:
# COCO classes
CLASSES = ['N/A', 'Person', 'Bicycle', 'Car', 'Motorcycle', 'Airplane', 'Bus', 
           'Train', 'Truck', 'Boat', 
           'Traffic-Light', 'Fire-Hydrant', 'N/A', 'Stop-Sign',
           'Parking Meter', 'Bench', 'Bird', 'Cat', 'Dog', 'Horse', 'Sheep', 'Cow', 'Elephant',
           'Bear', 'Zebra', 'Giraffe', 'N/A', 'Backpack', 'Umbrella', 'N/A', 'N/A', 'Handbag', 
           'Tie', 'Suitcase', 'Frisbee', 'Skis', 'Snowboard', 'Sports-Ball', 'Kite', 'Baseball Bat',
           'Baseball Glove', 'Skateboard', 'Surfboard', 'Tennis Racket', 'Bottle', 'N/A', 
           'Wine Glass', 'Cup', 'Fork', 'Knife', 'Spoon', 'Bowl', 'Banana', 'Apple', 'Sandwich', 
           'Orange', 'Broccoli', 'Carrot', 'Hot-Dog', 'Pizza', 'Donut', 'Cake', 'Chair', 'Couch', 
           'Potted Plant', 'Bed', 'N/A', 'Dining Table', 'N/A','N/A', 'Toilet', 'N/A', 'TV', 'Laptop',
           'Mouse', 'Remote', 'Keyboard', 'Cell-Phone', 'Microwave', 'Oven', 'Toaster', 
           'Sink', 'Refrigerator', 'N/A', 'Book', 'Clock', 'Vase', 'Scissors', 'Teddy-Bear',
           'Hair-Dryer', 'Toothbrush','plan']

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
len(CLASSES)

## The Bounding Box

The bounding boxes which are output from the model are of the form  $(x,y,w,h)$, where $(x,y)$ are in the units square $[0,1]^2$ and $h,w$ are the width and height of the bounding box. The coordinate $(x,y)$ represents the center of the object in the bounding box. In other words, the point $(x,y)$ is in the center of the bounding box. If the image is of size $W\times H$, then we can find the cordinates of the bounding box as follows:

$$
\begin{align}
\boxed{
x_0=W\cdot(x-w/2),\text{   }\text{  } y_0=H\cdot(y- h/2)}\\~\\
\boxed{
x_1=W\cdot(x +  w/2),\text{   }\text{  } y_1=H\cdot(y + h/2)}
\end{align}$$


In [ ]:

# Get coordinates (x0, y0, x1, y0) from model output (x, y, w, h)
def get_box_coords(boxes):
    x, y, w, h = boxes.unbind(1)
    x0, y0 = (x - 0.5 * w), (y - 0.5 * h)
    x1, y1 = (x + 0.5 * w), (y + 0.5 * h)
    box = [x0, y0, x1, y1]
    return torch.stack(box, dim=1)

### Scale Bounding Box Size

In [ ]:
# Scale box from [0,1] x [0,1] to [0, width] x [0, height]
def scale_boxes(output_box, width, height):
  box_coords = get_box_coords(output_box)
  scale_tensor = torch.Tensor([width, height, width, height]).to(torch.cuda.current_device())
  return box_coords * scale_tensor

## Detect Objects 

Now we need a function to encapsulate our object detection pipeline. The `detect` function returns the probability that objects belongs to the given classes, and the associated bounding boxes.

In [ ]:
def detect(im, model, transform):
    device = torch.cuda.current_device()
    width = im.size[0]
    height = im.size[1]
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0)
    img = img.to(device)

    # demo model only support by default images with aspect ratio between 0.5 and 2
    # if you want to use images with an aspect ratio outside this range
    # rescale your image so that the maximum size is at most 1333 for best results
    assert img.shape[-2] <= 1600 and img.shape[-1] <= 1600, 'demo model only supports images up to 1600 pixels on each side'

    # propagate through the model
    outputs = model(img)

    # keep only predictions with 0.7+ confidence
    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.85

    # convert boxes from [0; 1] to image scales
    bboxes_scaled = scale_boxes(outputs['pred_boxes'][0, keep], width, height)
    return probas[keep], bboxes_scaled

In [ ]:
probs,bboxes = detect(im, model, transform)
output=model(img_tensor.unsqueeze(0).to(torch.cuda.current_device()))

In [ ]:
output['pred_logits'].size()

In [ ]:
def plot_results(pil_img, prob, boxes,labels=True):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    for prob, (x0, y0, x1, y1), color in zip(prob, boxes.tolist(), COLORS * 100):
        ax.add_patch(plt.Rectangle((x0, y0), x1 - x0, y1 - y0,
                                   fill=False, color=color, linewidth=2))
        cl = prob.argmax()
        text = f'{CLASSES[cl]}: {prob[cl]:0.2f}'
        if labels:
          ax.text(x0, y0, text, fontsize=15,
                bbox=dict(facecolor=color, alpha=0.75))
    plt.axis('off')
    plt.show()

In [ ]:
plot_results(im, probs, bboxes)

### Without Labels

In [ ]:
plot_results(im, probs, bboxes, False)